In [ ]:
import pickle
import datetime
from pathlib import Path
import yaml
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sksurv.ensemble import RandomSurvivalForest
from sksurv.metrics import concordance_index_censored
from sksurv.util import Surv
from sklearn.inspection import permutation_importance
import optuna
import shap
from explainability import SHAP
from evaluation import evaluate_survival_model, PartialLogLikelihood
from training_survival_analysis import train_model
from models import MinimalisticNetwork
import matplotlib.pyplot as plt
import json

import torch
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, Dataset

In [ ]:
def prepare_rsf_data(df, time_col, event_col, random_state=1234, test_size=0.1, n_splits=5):
    """
    RSF 모델 학습을 위한 데이터 준비 함수.
    
    Args:
        df: 전처리된 데이터프레임 (결측치 없는 상태)
        time_col: 생존 시간을 나타내는 컬럼명 (ex: 'fu_total_yr')
        event_col: 생존 여부를 나타내는 컬럼명 (ex: 'survival')
        random_state: 재현성을 위한 랜덤 시드
        test_size: Train / test set split 비율. 0.1 디폴트
        n_splits: cross validation fold 수. 5 디폴트

    Returns:
        X_train, X_test, y_train, y_test, kfold
    """
    # 독립변수(X)에서 time_col, event_col 제거 후 One-Hot Encoding
    X = df.drop(columns=[time_col, event_col])
    X = pd.get_dummies(X, drop_first=True)

    # 종속변수(y)를 구조화 배열로 변환
    y = np.zeros(df.shape[0], dtype=[('vit_status', '?'), ('survival_time', '<f8')])
    y['vit_status'] = df[event_col].values.astype(bool)
    y['survival_time'] = df[time_col].values.astype(float)
    np.random.seed(random_state)

    # 데이터 분할 (Train/Test Split)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    # KFold 설정
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)

    return X_train, X_test, y_train, y_test, kfold

In [ ]:
def optimize_rsf(X_train, y_train, kfold, random_state=1234, n_trials=50):
    """
    Optuna를 사용해 Random Survival Forest의 최적 하이퍼파라미터를 찾는 함수.

    Args:
        X_train: 훈련 데이터 (독립변수)
        y_train: 훈련 데이터 (종속변수)
        kfold: K-Fold 객체
        random_state: 랜덤 시드
        n_trials: Optuna 하이퍼파라미터 서치 횟수

    Returns:
        best_params: 최적의 하이퍼파라미터
    """
    np.random.seed(random_state)
    # 설정 파일 로드
    config = yaml.safe_load(Path("./config.yaml").read_text())
    rsf_config = config["rsf"]
    
    # RSF의 하이퍼파라미터 `max_features`를 X_train의 column 수로 제한
    rsf_config["max_features"]["max"] = len(X_train.columns)
    model_name = "rsf"

    def objective_rsf(trial: optuna.Trial):
        """Optuna에서 사용할 목적 함수"""
        params = {
            "n_estimators": trial.suggest_int("n_estimators", rsf_config["n_estimators"]["min"], rsf_config["n_estimators"]["max"]),
            "min_samples_leaf": trial.suggest_int("min_samples_leaf", rsf_config["min_samples_leaf"]["min"], rsf_config["min_samples_leaf"]["max"]),
            "max_features": trial.suggest_int("max_features", rsf_config["max_features"]["min"], rsf_config["max_features"]["max"]),
            "max_depth": trial.suggest_int("max_depth", rsf_config["max_depth"]["min"], rsf_config["max_depth"]["max"]),
        } # config.yaml의 하이퍼파라미터 업로드
        scores = []
        for train_idx, _ in kfold.split(X_train, y_train): # 5-fold CV로 모델 학습 및 평가
            X_fold = X_train.iloc[train_idx]
            model = RandomSurvivalForest(n_estimators=params["n_estimators"],
                                        min_samples_leaf=params["min_samples_leaf"],
                                        max_features=params["max_features"],
                                        max_depth=params["max_depth"],
                                        random_state=random_state, n_jobs=-1)
            model.fit(X_fold, y_train[train_idx])
            score = model.score(X_fold, y_train[train_idx])
            scores.append(score)
        return np.mean(scores)

    # Optuna 실행 (하이퍼파라미터 탐색)
    study = optuna.create_study(study_name=model_name+str(datetime.datetime.now()),
                                direction="maximize",
                                sampler=optuna.samplers.TPESampler(seed=random_state))
    study.optimize(objective_rsf, n_trials=n_trials)
    best_params = study.best_trial.params

    return best_params

In [ ]:
def train_and_evaluate_rsf(X_train, X_test, y_train, y_test, best_params, kfold, random_state=1234):
    """
    Random Survival Forest를 학습하고 평가하는 함수.
    
    Args:
        X_train, X_test, y_train, y_test: 훈련 및 테스트 데이터
        best_params: Optuna에서 찾은 최적 하이퍼파라미터
        kfold: K-Fold 객체
        random_state: 랜덤 시드
    
    Returns:
        각 fold별 feature importance : .csv 파일로 저장, SHAP values beeswarm plot : .png파일로 저장 
        final_scores : 5-fold CV를 통해 계산한 모델의 최종 성능
    """
    np.random.seed(random_state)
    # 각 fold의 결과를 저장할 딕셔너리 생성
    fold_scores = {}
    
    # 최종 모델 학습 및 결과 저장
    for i, (train_fold, val_fold) in enumerate(kfold.split(X_train, y_train)):
        X_train_fold = X_train.iloc[train_fold]
        X_val_fold = X_train.iloc[val_fold]
        best_model = RandomSurvivalForest(**best_params, random_state=random_state, n_jobs=-1)
        best_model.fit(X_train_fold, y_train[train_fold])
        
        # 평가 결과 저장
        scores = evaluate_survival_model(best_model, X_val_fold, y_train[train_fold], y_train[val_fold])
        print(f"Final RSF Scores in Fold {i}: {scores}")
        fold_scores[f"fold_{i}"] = scores  
    
        # ---- Permutation Importance 저장 ----
        result = permutation_importance(best_model, X_val_fold, y_train[val_fold], n_repeats=15, random_state=random_state)
        result_dict = {k: result[k] for k in ("importances_mean", "importances_std")}
        permutation_importances = pd.DataFrame(result_dict, index=X_val_fold.columns).sort_values(by="importances_mean", ascending=False)
    
        perm_imp_path = f"RSF_permutation_importances_fold_{i}.csv"
        permutation_importances.to_csv(perm_imp_path, encoding = 'utf-8')
        print(f"Permutation importances saved to {perm_imp_path}")
        
        # ---- SHAP values 저장 ----
        explainer = shap.Explainer(best_model.predict, X_val_fold)
        shap_values = explainer(X_val_fold)
        
        # ---- SHAP Beeswarm Plot 저장 ----
        plt.figure(figsize=(10, 6))  # 적절한 크기 설정
        shap.plots.beeswarm(shap_values, show=False)
        
        beeswarm_path = f"RSF_beeswarm_fold_{i}.png"
        plt.tight_layout()  # 여백 자동 조정
        plt.savefig(beeswarm_path, dpi=300, bbox_inches="tight")  # 잘림 방지
        plt.close()  # 메모리 정리
        print(f"Beeswarm plot saved to {beeswarm_path}")
        
    # Final Scores
    fold_scores = pd.DataFrame(fold_scores).T
    final_scores = fold_scores.mean(skipna=True)
    print(final_scores)

    return final_scores

In [ ]:
############# 전처리 ################

df = pd.read_csv("processed_survival_data_modified.csv") # processed_survival_data_modified : Age 범주화한 데이터셋
df["implant_length_group"] = df["implant_length_group"].apply(
    lambda x: "Length ≥ 10" if x == "길이 10 이상" else "Length < 10"
)
df["survival"] = df["survival"].map({"survive": 0, "fail": 1}) # 종속변수(수술 성공 여부)를 0, 1로 변환
# 분석 제외할 변수 제거
exclude_columns = ["patient_ID", "me", "failure_reason", "failure_date", 
                   "last_fu_date", "surgery_Date", "fu_for_fail_yr", "fu_for_survival_yr"]
all_columns = [col for col in df.columns if col not in exclude_columns] # 분석에 사용할 변수만 포함
# 지정한 컬럼들에 결측치가 있는 행 제거
df = df.dropna(subset = all_columns)
df = df[all_columns]
selected_features = [col for col in df.columns if col not in ['fu_total_yr', 'survival'] + exclude_columns]
time_col = "fu_total_yr"
event_col = "survival"

In [ ]:
########## 최종 실행 코드 #############
X_train, X_test, y_train, y_test, kfold = prepare_rsf_data(df, "fu_total_yr", "survival")
best_params = optimize_rsf(X_train, y_train, kfold)
fold_scores_df = train_and_evaluate_rsf(X_train, X_test, y_train, y_test, best_params, kfold)
print(fold_scores_df)